In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import joblib

# =============================
# 🔹 Load Dataset
# =============================
file_path = 'dataset_sdn.csv'
df = pd.read_csv(file_path, encoding='latin1')

# Identify numeric and categorical features
numerical_features = [feature for feature in df.columns if df[feature].dtypes != 'O' and feature != "label"]
print("The number of numerical features is", len(numerical_features), "and they are : \n", numerical_features)
categorical_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
print("The number of categorical features is", len(categorical_features), "and they are : \n", categorical_features)
target = 'label'

# Separate features and target variable
X = df[numerical_features + categorical_features]
y = df[target]

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# =============================
# 🔹 Handle Missing Values
# =============================
# Impute missing values in numeric features
num_imputer = SimpleImputer(strategy='mean')
X_train_numeric = num_imputer.fit_transform(X_train[numerical_features])
X_test_numeric = num_imputer.transform(X_test[numerical_features])

# Impute missing values in categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = cat_imputer.fit_transform(X_train[categorical_features])
X_test_categorical = cat_imputer.transform(X_test[categorical_features])

# =============================
# 🔹 Standardize Numeric Features
# =============================
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# =============================
# 🔹 Encode Categorical Features
# =============================
vectorizer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = vectorizer.fit_transform(X_train_categorical)
X_test_cat_encoded = vectorizer.transform(X_test_categorical)

# =============================
# 🔹 Combine Processed Features
# =============================
X_train_final = np.hstack((X_train_numeric_scaled, X_train_cat_encoded))
X_test_final = np.hstack((X_test_numeric_scaled, X_test_cat_encoded))

# Convert target to NumPy arrays
y_train = y_train.values
y_test = y_test.values

# =============================
# 🔹 Helper Function for Evaluation
# =============================
def evaluate_model(y_true, y_pred, model_name):
    print(f"Evaluation for {model_name}:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))
    conf_matrix = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    print("\n")


# =============================
# 🔹 Random Forest
# =============================
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_final, y_train)
y_pred_rf = rf_model.predict(X_test_final)
evaluate_model(y_test, y_pred_rf, "Random Forest")

# Save Random Forest model
joblib.dump(rf_model, 'Modells/RandomForest_model.pkl')
joblib.dump(scaler, 'Modells/RandomForest_scaler.pkl')
joblib.dump(vectorizer, 'Modells/RandomForest_vectorizer.pkl')

print("Random Forest model, scaler, and vectorizer saved.")

The number of numerical features is 19 and they are : 
 ['dt', 'switch', 'pktcount', 'bytecount', 'dur', 'dur_nsec', 'tot_dur', 'flows', 'packetins', 'pktperflow', 'byteperflow', 'pktrate', 'Pairflow', 'port_no', 'tx_bytes', 'rx_bytes', 'tx_kbps', 'rx_kbps', 'tot_kbps']
The number of categorical features is 3 and they are : 
 ['src', 'dst', 'Protocol']
Evaluation for Random Forest:
Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12712
           1       1.00      1.00      1.00      8157

    accuracy                           1.00     20869
   macro avg       1.00      1.00      1.00     20869
weighted avg       1.00      1.00      1.00     20869

TP: 8157, TN: 12712, FP: 0, FN: 0


Random Forest model, scaler, and vectorizer saved.
